In [105]:
# https://account.sendinblue.com/advanced/api

from __future__ import print_function
import time
import requests
import sib_api_v3_sdk
from sib_api_v3_sdk.rest import ApiException
from pprint import pprint

import pandas as pd
import csv

from datetime import datetime

# chargement de la clef API via dotenv

from dotenv import load_dotenv
load_dotenv()
SIB_API_KEY = os.environ.get('SIB_API_KEY')
DATA_FOLDER = os.environ.get('DATA_FOLDER')

In [4]:
# Get all the contacts : https://developers.sendinblue.com/reference/getcontacts-1

def get_all_contacts(apikey, limit = 50, offset = 0, modified_since = '2020-09-20T19:20:30+01:00', sort = 'desc'):
    """Récupère l'intégralité des contacts du compte

    Parameters
    ----------
    apikey : str
        API key
    limit : int64
        Number of documents per page
    offset : int64
        Index of the first document in the page
    modified_since : string (UTC date-time : 'YYYY-MM-DDTHH:mm:ss.SSSZ')
        Filter (urlencoded) the contacts modified after a given UTC date-time (YYYY-MM-DDTHH:mm:ss.SSSZ).
    sort : str ('asc' or 'desc')
        Sort the results in the ascending/descending order of record creation. Default order is descending if sort is not passed

    Returns
    -------
    contacts : Json
        Liste de tous les contacts correspondant à la requête

    """
    configuration = sib_api_v3_sdk.Configuration()
    configuration.api_key['api-key'] = apikey
    api_instance = sib_api_v3_sdk.ContactsApi(sib_api_v3_sdk.ApiClient(configuration))
    try:
        api_response = api_instance.get_contacts(limit=limit, offset=offset, modified_since=modified_since, sort = sort)
        pprint(api_response)
    except ApiException as e:
        print("Exception when calling ContactsApi->get_contacts: %s\n" % e)


get_all_contacts(SIB_API_KEY, modified_since = '2022-01-06T00:00:00+01:00')

{'contacts': [{'attributes': {'NOM': 'John',
                              'PRENOM': 'Doe',
                              'SMS': '31234567923'},
               'createdAt': '2022-01-10T21:25:07.396+01:00',
               'email': '#john@example.com',
               'emailBlacklisted': False,
               'id': 239,
               'listIds': [15, 16],
               'modifiedAt': '2022-01-12T17:01:39.032+01:00',
               'smsBlacklisted': False},
              {'attributes': {'NOM': 'Turcato', 'PRENOM': 'Nicolas'},
               'createdAt': '2022-01-04T17:22:19.000+01:00',
               'email': 'nicolas.turcato@etudiant.univ-lr.fr',
               'emailBlacklisted': False,
               'id': 238,
               'listIds': [15],
               'modifiedAt': '2022-01-12T17:01:39.032+01:00',
               'smsBlacklisted': False},
              {'attributes': {'NOM': 'Bonneau', 'PRENOM': 'Yohann'},
               'createdAt': '2022-01-04T17:21:44.000+01:00',
               

In [98]:
# Get all the lists : https://developers.sendinblue.com/reference/getlists-1

def get_all_lists(apikey, limit = 50, offset = 0):
    """Récupère l'intégralité des listes du compte

    Parameters
    ----------
    apikey : str
        API key
    limit : int64
        Number of documents per page
    offset : int64
        Index of the first document in the page
    sort : str ('asc' or 'desc')
        Sort the results in the ascending/descending order of record creation. Default order is descending if sort is not passed

    Returns
    -------
    count : int64
        Nombre de listes
    lists : Json
        Liste de toutes les listes suivant la requête

    """
    configuration = sib_api_v3_sdk.Configuration()
    configuration.api_key['api-key'] = apikey
    api_instance = sib_api_v3_sdk.ListsApi(sib_api_v3_sdk.ApiClient(configuration))

    try:
        api_response = api_instance.get_lists(limit=limit, offset=offset)
        pprint(api_response)
    except ApiException as e:
        print("Exception when calling ListsApi->get_lists: %s\n" % e)

get_all_lists(SIB_API_KEY)



{'count': 15,
 'lists': [{'folderId': 13,
            'id': 19,
            'name': 'manif_interet_univ',
            'totalBlacklisted': 0,
            'totalSubscribers': 0,
            'uniqueSubscribers': 10},
           {'folderId': 11,
            'id': 18,
            'name': 'testeurs_prospects',
            'totalBlacklisted': 0,
            'totalSubscribers': 0,
            'uniqueSubscribers': 9},
           {'folderId': 1,
            'id': 17,
            'name': 'testeurs_vip',
            'totalBlacklisted': 0,
            'totalSubscribers': 0,
            'uniqueSubscribers': 6},
           {'folderId': 1,
            'id': 16,
            'name': 'relance_post_onboarding',
            'totalBlacklisted': 0,
            'totalSubscribers': 0,
            'uniqueSubscribers': 32},
           {'folderId': 1,
            'id': 15,
            'name': 'nouveaux_arrivants',
            'totalBlacklisted': 0,
            'totalSubscribers': 0,
            'uniqueSubscribers

In [ ]:
# Import contacts : https://developers.sendinblue.com/reference/importcontacts-1

# default listIds = "nouveau_contact"

def import_contacts(apikey, fileBody, listIds):
    """Récupère l'intégralité des listes du compte

    Parameters
    ----------
    apikey : str
        API key
    fileBody : .txt or .csv
        Mandatory if fileUrl is not defined. CSV content to be imported.
        Use semicolon to separate multiple attributes.
        Maximum allowed file body size is 10MB.
        However we recommend a safe limit of around 8 MB to avoid the issues caused due to increase of file body size while parsing.
        Please use fileUrl instead to import bigger files.
    listIds : array of integers
        Mandatory if newList is not defined. Ids of the lists in which the contacts shall be imported.
        For example, [2, 4, 7].

    Returns
    -------

    """

    configuration = sib_api_v3_sdk.Configuration()
    configuration.api_key['api-key'] = apikey

    api_instance = sib_api_v3_sdk.ContactsApi(sib_api_v3_sdk.ApiClient(configuration))
    request_contact_import = sib_api_v3_sdk.RequestContactImport()
    request_contact_import.file_body = fileBody
    request_contact_import.list_ids = listIds
    request_contact_import.email_blacklist = False
    request_contact_import.sms_blacklist = False
    request_contact_import.update_existing_contacts = True
    request_contact_import.empty_contacts_attributes = False

    try:
        api_response = api_instance.import_contacts(request_contact_import)
        pprint(api_response)
    except ApiException as e:
        print("Exception when calling ContactsApi->import_contacts: %s\n" % e)

fileBodyTest = "EMAIL;NOM;PRENOM;SMS\n#john@example.com;John;Doe;31234567923"

import_contacts(SIB_API_KEY, fileBody = fileBodyTest , listIds=[15])


In [75]:
# Save list of all lists to csv

configuration = sib_api_v3_sdk.Configuration()
configuration.api_key['api-key'] = SIB_API_KEY

api_instance = sib_api_v3_sdk.ListsApi(sib_api_v3_sdk.ApiClient(configuration))
limit = 10
offset = 0

try:
    api_response = api_instance.get_lists(limit=limit, offset=offset)
    # pprint(api_response)
except ApiException as e:
    print("Exception when calling ListsApi->get_lists: %s\n" % e)

list_of_lists = api_response.lists

list_file_path = os.path.join('/home/sy/Téléchargements', 'sendinblue_lists' + '.csv')

keys = list_of_lists[0].keys()

with open(list_file_path, 'w', newline='') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(list_of_lists)

In [ ]:
# Add a contact to a list : https://developers.sendinblue.com/reference/addcontacttolist-1

configuration = sib_api_v3_sdk.Configuration()
configuration.api_key['api-key'] = SIB_API_KEY

api_instance = sib_api_v3_sdk.ListsApi(sib_api_v3_sdk.ApiClient(configuration))
list_id = 16
contact_emails = sib_api_v3_sdk.AddContactToList()
contact_emails.emails = ['#john@example.com']

try:
    api_response = api_instance.add_contact_to_list(list_id, contact_emails)
    pprint(api_response)
except ApiException as e:
    print("Exception when calling ListsApi->add_contact_to_list: %s\n" % e)

In [ ]:
# Delete a contact from a list : https://developers.sendinblue.com/reference/removecontactfromlist

configuration = sib_api_v3_sdk.Configuration()
configuration.api_key['api-key'] = SIB_API_KEY

api_instance = sib_api_v3_sdk.ListsApi(sib_api_v3_sdk.ApiClient(configuration))
list_id = 16
contact_emails = sib_api_v3_sdk.RemoveContactFromList()
contact_emails.emails = ['#john@example.com']

try:
    api_response = api_instance.remove_contact_from_list(list_id, contact_emails)
    pprint(api_response)
except ApiException as e:
    print("Exception when calling ListsApi->remove_contact_from_list: %s\n" % e)

In [ ]:
# Get all folders : https://developers.sendinblue.com/reference/getfolders-1

configuration = sib_api_v3_sdk.Configuration()
configuration.api_key['api-key'] = SIB_API_KEY

api_instance = sib_api_v3_sdk.ContactsApi(sib_api_v3_sdk.ApiClient(configuration))
limit = 10
offset = 0

try:
    api_response = api_instance.get_folders(limit, offset)
    pprint(api_response)
except ApiException as e:
    print("Exception when calling ContactsApi->get_folders: %s\n" % e)



In [89]:
# charger le dernier CSV

df = pd.read_csv(os.path.join(DATA_FOLDER, 'inscriptions20220114.csv'))

print("Number of records in the current sendinblue contact list : " + str(len(all_contacts)))
print("Number of records in the up-to-date list : " + str(len(df)))
print("Matching emails between sendinblue and complete list : " + str(len(df[df['mail'].isin(all_contacts)])))
print("Non-matching emails between sendinblue and complete list : " + str(len(df[~df['mail'].isin(all_contacts)])))

# print("Matching emails between sendinblue and complete liste : " + str(len(df[df['mail'].isin(all_contacts)])))
# print("Non-matching emails between sendinblue and complete liste : " + str(len(~df[df['mail'].isin(all_contacts)]))

Number of records in the current sendinblue contact list : 236
Number of records in the up-to-date list : 216
Matching emails between sendinblue and complete list : 194
Non-matching emails between sendinblue and complete list : 22


In [83]:
df[~df['mail'].isin(all_contacts)].sort_values(by = 'inscription_time')

# bug : il y a des gens qui sont là depuis très longtemps
# filtrer par date ? Last modified ?

,id,inscription_time,prenom,nom,mail,orga
140,7824712,2021-03-02T07:58:00Z,Marie-Virginie,CONNAC,Connacm@excelia-group.com,L'école de commerce Excelia
179,8167231,2021-03-30T16:57:00Z,Romain,LE GALL,romainbenjamin.lg@gmail.com,NaN
99,8495196,2021-05-03T06:36:00Z,Joël,LAGNEAU,Lagneauj19@excelia-group.com,L'école de commerce Excelia
123,8495231,2021-05-03T06:38:00Z,Lisa,BERTUCCI,bertuccil18@excelia-group.com,L'école de commerce Excelia
178,8495762,2021-05-03T07:20:00Z,Romain,ALLINE,alliner16@excelia-group.com,La Communauté d'Agglomération de La Rochelle
22,8495920,2021-05-03T07:31:00Z,Camille,DUPUY,Dupuyc18@excelia-group.com,L'école de commerce Excelia
57,9071332,2021-06-24T06:52:00Z,Eric,VILLETTE,Eric.villette@ville-larochelle.fr,La Ville de La Rochelle
193,9423991,2021-09-03T09:18:00Z,Tester,TEST,test@gmail.com,La Communauté d'Agglomération de La Rochelle
44,9552449,2021-09-16T07:37:00Z,Clément,BRANGER,Brangerc19@excelia-group.com,L'école de commerce Excelia
157,9556074,2021-09-16T11:45:00Z,Néomie,MVUAZI,Neomimvuazi@gmail.com,L'école de commerce Excelia


In [104]:
## Recipe 1 : uploader les nouveaux contacts dans la liste "nouveaux_arrivants"

# 1 - récupérer les mails des contacts dans la base SIB

def get_all_sib_mails(apikey, limit = 500, offset = 0, modified_since = '2020-09-20T19:20:30+01:00', sort = 'desc'):
    """Récupère l'intégralité des mails des contacts du compte SIB sous forme d'une liste

    Parameters
    ----------
    apikey : str
        API key
    limit : int64
        Number of documents per page
    offset : int64
        Index of the first document in the page
    modified_since : string (UTC date-time : 'YYYY-MM-DDTHH:mm:ss.SSSZ')
        Filter (urlencoded) the contacts modified after a given UTC date-time (YYYY-MM-DDTHH:mm:ss.SSSZ).
    sort : str ('asc' or 'desc')
        Sort the results in the ascending/descending order of record creation. Default order is descending if sort is not passed

    Returns
    -------
    all_contacts : List
        Liste des mails des contacts correspondant à la requête

    """
    configuration = sib_api_v3_sdk.Configuration()
    configuration.api_key['api-key'] = apikey
    api_instance = sib_api_v3_sdk.ContactsApi(sib_api_v3_sdk.ApiClient(configuration))
    try:
        api_response = api_instance.get_contacts(limit=limit, offset=offset, modified_since=modified_since, sort = sort)
        all_contacts = [d['email'] for d in api_response.contacts]
        return(all_contacts)
    except ApiException as e:
        print("Exception when calling ContactsApi->get_contacts: %s\n" % e)

all_sib_mails = get_all_sib_mails(SIB_API_KEY)

# 2 - filtrer la base à jour pour ne retenir que les non match sur la base des mails & 3 - convertir la base à jour filtré en scsv

new_contacts = df[~df['mail'].isin(all_sib_mails)].to_csv(sep = ";")

## WARNING : peut-être filtrer pour ne retenir que les inscriptions post-20220101

# 4 - uploader à la liste "nouveaux_arrivants" [15]

# import_contacts(SIB_API_KEY, fileBody = new_contacts , listIds=[15])



TypeError: string indices must be integers